In [3]:
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments
import torch
import pandas as pd
from sklearn.model_selection import train_test_split

import os

# Отключаем W&B
os.environ["WANDB_DISABLED"] = "true"

# Подготовка данных
data = pd.DataFrame({
    'text': [
        "Текст 1: Сегодня погода солнечная и теплая. -- Текст 2: Планирую погулять в парке. Является продолжением? [да/нет]:", 
        "Текст 1: Вчера шел дождь, и мы остались дома. -- Текст 2: Завтра я планирую купить продукты. Является продолжением? [да/нет]:",
        "Текст 1: Он купил билеты на самолет. -- Текст 2: Он с нетерпением ждал своей поездки. Является продолжением? [да/нет]:", 
        "Текст 1: Мы были на концерте вчера вечером. -- Текст 2: Сегодня я снова пойду на концерт. Является продолжением? [да/нет]:",
        "Текст 1: Она сделала домашку за вечер. -- Текст 2: У нее не было времени на другие дела. Является продолжением? [да/нет]:", 
        "Текст 1: Я купил новую книгу по программированию. -- Текст 2: Я начал изучать новый язык программирования. Является продолжением? [да/нет]:",
        "Текст 1: Он выиграл соревнования по шахматам. -- Текст 2: Он готовится к следующему турниру. Является продолжением? [да/нет]:", 
        "Текст 1: Мы отпраздновали Новый год с друзьями. -- Текст 2: В следующем году мы планируем провести праздник вместе. Является продолжением? [да/нет]:", 
        "Текст 1: Я занимаюсь спортом каждое утро. -- Текст 2: Сегодня я пропустил тренировку. Является продолжением? [да/нет]:",
        "Текст 1: Вчера я потерял свой телефон. -- Текст 2: Сегодня мне удалось его найти. Является продолжением? [да/нет]:",
        "Текст 1: Мы поехали на отдых на море. -- Текст 2: Мы планируем вернуться туда летом. Является продолжением? [да/нет]:", 
        "Текст 1: Я начал читать новую книгу по истории. -- Текст 2: Эта книга рассказывает о древних цивилизациях. Является продолжением? [да/нет]:"
    ],
    'label': [1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0]  # 1 - продолжение, 0 - не продолжение
})

train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# Загрузка токенизатора и модели
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3medium_based_on_gpt2")
model = GPT2ForSequenceClassification.from_pretrained("sberbank-ai/rugpt3medium_based_on_gpt2", num_labels=2)

# Токенизация
# Токенизация
def tokenize_function(examples):
    return tokenizer(list(examples["text"]), padding="max_length", truncation=True, max_length=128)

train_encodings = tokenize_function(train_data)
val_encodings = tokenize_function(val_data)


# Подготовка датасетов
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = Dataset(train_encodings, train_data['label'].tolist())
val_dataset = Dataset(val_encodings, val_data['label'].tolist())

# Настройка тренировки
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    logging_steps=10,
    logging_first_step=True, 
    learning_rate=2e-5,
    per_device_train_batch_size=2,  
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    #report_to="none",
)

# Создание Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Тренировка
trainer.train()

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at sberbank-ai/rugpt3medium_based_on_gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss
1,1.043200,0.532534
2,0.658500,0.544886
3,0.658500,0.703137


TrainOutput(global_step=15, training_loss=0.6941921869913737, metrics={'train_runtime': 22.8694, 'train_samples_per_second': 1.181, 'train_steps_per_second': 0.656, 'total_flos': 6268772155392.0, 'train_loss': 0.6941921869913737, 'epoch': 3.0})